# ID2214/FID3214 Assignment 2 Group no. 12
### Project members: 
Aksel Uhr, auhr@kth.se
Olivia Höft, hoft@kth.se
Ilias Merentitis, iliasme@kth.se

### Declaration:
By submitting this solution, it is hereby declared that all individuals listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214/FID3214) and no part of the solution has been provided by someone not listed as project member above.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy, pandas and time may be used in the solution for this assignment.

### Instructions
All parts of the assignment starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of all parts of the assignment starting 
with number 2 below, then the assignment will receive 2 points (in total).

Note that you do not have to develop the code directly within the notebook
but may instead copy the comments and test cases to a more convenient development environment
and when everything works as expected, you may paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above).

## Load NumPy, pandas and time

In [ ]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt

## Reused functions from Assignment 1

In [ ]:
# Copy and paste functions from Assignment 1 here that you need for this assignment

def create_column_filter(df):
    
    # Copies the input df
    dfCopy = df.copy()  
   
    # Drops each column with only one unique value + one unique value with nan(s)
    for col in dfCopy.columns:
        if (col != "CLASS" and col != "ID") and (len(dfCopy[col].unique()) == 1 or (len(dfCopy[col].unique()) == 2 and dfCopy.isnull().values.any())):
            dfCopy.drop(col,inplace=True,axis=1)
             
    # Extracting column names and converts to list
    remainingColumns = dfCopy.columns.values.tolist()
    return dfCopy, remainingColumns

def apply_column_filter(df, column_filter):
    
    # Copies the input df
    dfCopy = df.copy()
    
    # Finds all similar column names and keep the values of them
    # Discards the rest
    df_final = dfCopy[dfCopy.columns.intersection(column_filter)]
    return df_final

def create_normalization(df, normalizationtype = "minmax"):
  
    # Creates df copy
    dfCopy = df.copy()
    
    # Empty dict for normalization
    normalization = {}

    # Iterating through the columns
    for col in dfCopy.columns:
        
        # Checks type and column name
        if (col != "CLASS" and col != "ID") and (dfCopy[col].dtype == np.int64 or dfCopy[col].dtype == np.float64):
            
            # Fetches min and max for each column and applies normalization
            # Adds to dictionary
            if(normalizationtype == "minmax"):
                min = dfCopy[col].min()
                max = dfCopy[col].max()
                dfCopy[col] = [(x-min)/(max-min) for x in dfCopy[col]]
                normalization[col] = ("minmax", min, max)
                
            # Calculates mean and standard deviation for each column and applies noralization
            # Adds to dictionary
            elif (normalizationtype == "zscore"):
                mean = dfCopy[col].mean()
                std = dfCopy[col].std()
                dfCopy[col] = df[col].apply(lambda x: (x-mean)/std)
                normalization[col] = ("zscore",mean,std)

    return dfCopy, normalization

def apply_normalization(df, normalization):
    
    # Creates df copy
    dfCopy = df.copy()
    
    # Applies different normalization based on type for each column 
    for col in dfCopy:
        if(col != "CLASS" and col != "ID"):
            if(normalization[col][0] == 'minmax'):
                dfCopy[col] = [(x-normalization[col][1])/(normalization[col][2]-normalization[col][1]) for x in dfCopy[col]]
            elif(normalization[col][0] == 'zscore'):
                dfCopy[col].apply(lambda x: (x- normalization[col][1])/normalization[col][2])
            
    return dfCopy    

def create_imputation(df):
    dfCopy = df.copy()
    dictionary = {}
    
    for col in dfCopy:
        
        # Checks type and column name
        if (col != "CLASS" and col != "ID") and (dfCopy[col].dtype == np.int64 or dfCopy[col].dtype == np.float64):
                if dfCopy[col].isnull().all():
                    dfCopy[col].fillna(0, inplace=True)
                    dictionary[col] = 0 
                else:
                    mean = dfCopy[col].mean()
                    dfCopy[col].fillna(mean, inplace=True)
                    dictionary[col] = mean 
        elif dfCopy[col].dtype == np.object:
                if dfCopy[col].isnull().all():
                    dfCopy[col].fillna('', inplace=True)
                    dictionary[col] = '' 
                else:
                    mode = dfCopy[col].mode()
                    dfCopy[col].fillna(mode[0], inplace=True)
                    dictionary[col] = mode[0] 
                    
    return dfCopy, dictionary

def apply_imputation(df, dictionary):
    dfCopy = df.copy()
    
    for col in dictionary:
        dfCopy.fillna(dictionary[col], inplace=True)
        
    return dfCopy

def create_bins(df, nobins = 10, bintype = "equal-width"):
    dfCopy = df.copy()
    binning = {}

    for col in dfCopy.columns:
        if (col != "CLASS" and col != "ID") and (dfCopy[col].dtype == np.int64 or dfCopy[col].dtype == np.float64):
            #Buckets with approximately the same range
            if bintype == "equal-width":
                dfCopy[col], bins = pd.cut(dfCopy[col], nobins, retbins=True, labels=False)
                binning[col] = bins
            #Buckets with approximately the same number of values in each bucket
            elif bintype == "equal-size":
                dfCopy[col], bins = pd.qcut(dfCopy[col], q=nobins, retbins=True, labels=False, duplicates="drop")
                binning[col] = bins

            binning[col][0] = -np.inf
            binning[col][-1] = np.inf

    dfCopy[dfCopy.columns] = dfCopy[dfCopy.columns].astype('category')
    return dfCopy, binning

def apply_bins(df, binning):
    dfCopy = df.copy()

    for col in binning:
        dfCopy[col] = pd.cut(dfCopy[col], binning[col], labels=False)

    dfCopy[dfCopy.columns] = dfCopy[dfCopy.columns].astype('category')
    return dfCopy

def accuracy(df, correctlabels):
    dfCopy = df.copy()
    
    highestLabels = dfCopy.idxmax(axis=1)
    highestLabelsList = highestLabels.values.tolist()
    
    count = 0
    dfLength = len(dfCopy)
    
    for i in range(len(correctlabels)):
        if correctlabels[i] == highestLabelsList[i]:
            count = count + 1
    
    res = count / dfLength
    
    return res

def brier_score(df, correctlabels):
    dfCopy = df.copy()
    
    correctDf = pd.get_dummies(correctlabels)
    
    brier_score = np.mean(np.sum((dfCopy-correctDf)**2, axis= 1))
    
    return brier_score

def get_pos_and_neg(predictions, pos_label, correct_labels):
    scores = {}
    
    # Counts tps amd fps and put them in a dict
    for i in range(len(predictions)):
        score = predictions[pos_label][i] 
        (p,n) = scores.get(score,(0,0))
        if correct_labels[i] == pos_label:
            scores[score] = (p+1,n)
        else:
            scores[score] = (p,n+1)

    # dict to array
    # sort array
    scores = np.array([[score,scores[score][0],scores[score][1]] for score in scores.keys()])
    scores = scores[np.argsort(scores[:,0])[::-1]]
    
    # drop predictions
    pos = scores[:,1]
    neg = scores[:,2]
    
    return pos, neg


def auc(predictions, correctlabels):
    
    AUC = 0
    
    for col in predictions.columns:
        
        #Räknar ut antal labels i correct labels
        class_frequency = correctlabels.value_counts()[col]
        
        # Gets total true positives and true negatives for each column
        pos, neg = get_pos_and_neg(predictions,col, correctlabels)
        
        # Area for each column
        thisAUC = calculateAUC(neg, pos, 0)
        
        # Total area for the predictions
        # "How many labels (e.g. A) do we have in correct labels (in 
        # This step considers ALL classes and calculates tot AUC, considering each column
        AUC += thisAUC*(class_frequency/len(correctlabels))

    return AUC
    
    
def calculateAUC(neg, pos, AUC):
    Tot_tp = sum(pos)
    Tot_fp = sum(neg)
    
    Cov_tp = 0
    for i in range(len(pos)):
        
        # Cov tp holds true positives
        AUC += (Cov_tp/Tot_tp)*(neg[i]/Tot_fp)+ (pos[i]/Tot_tp)*(neg[i]/Tot_fp)/2
        Cov_tp += pos[i]

    return AUC



## 1. Define the class kNN

In [138]:
# Define the class kNN with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, imputation, normalization, one_hot, labels, training_labels, training_data, training_time
#
# Input to fit:
# self              - the object itself
# df                - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# normalizationtype - "minmax" (default) or "zscore"
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter   - a column filter (see Assignment 1) from df
# self.imputation      - an imputation mapping (see Assignment 1) from df
# self.normalization   - a normalization mapping (see Assignment 1), using normalizationtype from the imputed df
# self.one_hot         - a one-hot mapping (see Assignment 1)
# self.training_labels - a pandas series corresponding to the "CLASS" column, set to be of type "category" 
# self.labels          - a list of the categories (class labels) of the previous series
# self.training_data   - the values (an ndarray) of the transformed dataframe, i.e., after employing imputation, 
#                        normalization, and possibly one-hot encoding, and also after removing the "CLASS" and "ID" columns
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# k    - an integer >= 1 (default = 5)
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are estimated by the relative class frequencies in the set of class labels from the k nearest 
#               (with respect to Euclidean distance) neighbors in training_data
#
# Hint 1: Drop any "CLASS" and "ID" columns first and then apply column filtering, imputation, normalization and one-hot
#
# Hint 2: Get the numerical values (as an ndarray) from the resulting dataframe and iterate over the rows 
#         calling some sub-function, e.g., get_nearest_neighbor_predictions(x_test,k), which for a test row
#         (numerical input feature values) finds the k nearest neighbors and calculate the class probabilities.
#
# Hint 3: This sub-function may first find the distances to all training instances, e.g., pairs consisting of
#         training instance index and distance, and then sort them according to distance, and then (using the indexes
#         of the k closest instances) find the corresponding labels and calculate the relative class frequencies

class kNN:

    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.normalization = None
        self.one_hot = None
        self.labels = None 
        self.training_labels = None 
        self.training_data = None 
        self.training_time = None

    def fit(self, df, normalizationtype = "minmax"):
        
        df, self.column_filter = create_column_filter(df)
        df, self.imputation = create_imputation(df)
        df, self.normalization = create_normalization(df, normalizationtype)
        self.training_labels = df["CLASS"].astype("category")
        self.labels = pd.value_counts(df.CLASS).index.tolist() 
        self.training_data = df.drop(columns = ["CLASS", "ID"]).to_numpy()

        
    def get_nearest_neighbor_predictions(self, x_test, k):
        
        argSortedDistance = np.argsort([self.eucledean_distance(x_test,x_train) for x_train in self.training_data], axis=0)
        
        nearestNeighbors = argSortedDistance[:k]
        
        labels = self.training_labels[nearestNeighbors]
        
        probabilities = labels.value_counts(normalize = True)
        return probabilities
        
        
    def predict(self, df, k):
        dfCopy = df.copy();
        
        dfCopy.drop(columns = ["CLASS", "ID"], inplace=True)
        dfCopy = apply_column_filter(dfCopy, self.column_filter)
        dfCopy = apply_imputation(dfCopy, self.imputation)
        dfCopy = apply_normalization(dfCopy, self.normalization)
        
        testValues = dfCopy.to_numpy()
        
        dfPredictions = pd.DataFrame(columns = np.unique(self.training_labels))
        
        for i in range(len(testValues)):
            prob = self.get_nearest_neighbor_predictions(testValues[i], k)            
            dfProb = pd.DataFrame([prob])
            dfPredictions = pd.concat([dfPredictions, dfProb], ignore_index=True)
        
        return dfPredictions
    
    def eucledean_distance(self, x, y):
        return np.sqrt(np.sum((x-y)**2, axis=0));
        
        
        

In [139]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.csv")

glass_test_df = pd.read_csv("glass_test.csv")

knn_model = kNN()

t0 = time.perf_counter()
knn_model.fit(glass_train_df)
print(kNN)

print("Training time: {0:.2f} s.".format(time.perf_counter()-t0))

test_labels = glass_test_df["CLASS"]
print(kNN)

k_values = [1,3,5,7,9]
results = np.empty((len(k_values),3))

for i in range(len(k_values)):
    t0 = time.perf_counter()
    predictions = knn_model.predict(glass_test_df,k=k_values[i])
    print("Testing time (k={0}): {1:.2f} s.".format(k_values[i],time.perf_counter()-t0))
    
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=k_values,columns=["Accuracy","Brier score","AUC"])

print()
display("results",results)

<ipython-input-120-7ad9a31da64c>:89: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif dfCopy[col].dtype == np.object:


<class '__main__.kNN'>
Training time: 0.01 s.
<class '__main__.kNN'>
Testing time (k=1): 0.59 s.
Testing time (k=3): 0.64 s.
Testing time (k=5): 0.79 s.
Testing time (k=7): 0.66 s.
Testing time (k=9): 0.71 s.



'results'

,Accuracy,Brier score,AUC
1,0.747664,0.504673,0.810350
3,0.663551,0.488058,0.815859
5,0.579439,0.474019,0.833805
7,0.598131,0.470723,0.834465
9,0.616822,0.483674,0.828734


In [140]:
train_labels = glass_train_df["CLASS"]
predictions = knn_model.predict(glass_train_df,k=1)
print("Accuracy on training set (k=1): {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set (k=1): {0:.4f}".format(auc(predictions,train_labels)))
print("Brier score on training set (k=1): {0:.4f}".format(brier_score(predictions,train_labels)))

Accuracy on training set (k=1): 1.0000
AUC on training set (k=1): 1.0000
Brier score on training set (k=1): 0.0000


### Comment on assumptions, things that do not work properly, etc.


## 2. Define the class NaiveBayes

In [135]:
# Define the class NaiveBayes with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, binning, labels, class_priors, feature_class_value_counts, feature_class_counts
#
# Input to fit:
# self    - the object itself
# df      - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# nobins  - no. of bins (default = 10)
# bintype - either "equal-width" (default) or "equal-size" 
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter              - a column filter (see Assignment 1) from df
# self.binning                    - a discretization mapping (see Assignment 1) from df
# self.class_priors               - a mapping (dictionary) from the labels (categories) of the "CLASS" column of df,
#                                   to the relative frequencies of the labels
# self.labels                     - a list of the categories (class labels) of the "CLASS" column of df
# self.feature_class_value_counts - a mapping from the feature (column name) to the number of
#                                   training instances with a specific combination of  
#                                   value for the feature and class label
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are estimated by the naive approximation of Bayes rule (see lecture slides)
#
# Hint 1: First apply the column filter and discretization
#
# Hint 2: Iterating over either columns or rows, and for each possible class label, calculate the relative
#         frequency of the observed feature value given the class (using feature_class_value_counts and 
#         feature_class_counts) 
#
# Hint 3: Calculate the non-normalized estimated class probabilities by multiplying the class priors to the
#         product of the relative frequencies
#
# Hint 4: Normalize the probabilities by dividing by the sum of the non-normalized probabilities; in case
#         this sum is zero, then set the probabilities to the class priors

class NaiveBayes:

    def __init__(self):
        
        self.column_filter = None
        self.binning = None
        self.class_priors = None 
        self.labels = None 
        self.feature_class_value_counts = None 
        self.feature_class_counts = None

    def fit(self, df, nobins = 10, bintype = "equal-width"):
        
        df, self.column_filter = create_column_filter(df)
        dfCopy, self.binning = create_bins(df, nobins, bintype)
        self.class_priors = df["CLASS"].value_counts(normalize = True).to_dict()
        self.labels = np.unique(df["CLASS"])
        self.feature_class_value_counts = {}
   
        feature_class_value_counts = dict()
        for classLabel in self.labels:
            feature_class_value_counts[classLabel] = dict()
            for col in dfCopy.columns:
                if(col != "CLASS" and col != "ID"):
                    currClass = dfCopy.loc[dfCopy["CLASS"] == classLabel]
                    test = currClass[col].value_counts(normalize=True, sort = False)
                    feature_class_value_counts[classLabel][col] = test
            
        self.feature_class_value_counts = feature_class_value_counts
        
    def predict(self, df):
        dfCopy = df.copy()
        dfCopy = apply_column_filter(dfCopy, self.column_filter)
        dfCopy = apply_bins(dfCopy, self.binning)
        
        dfResults = pd.DataFrame(columns=np.unique(dfCopy["CLASS"]))
        
        dfCopy = dfCopy.drop(["CLASS", "ID"], axis = 1)
        
        #iterate over the rows in the Dataframe
        for index, row in dfCopy.iterrows():
            class_prob = {}
            newList = {}
            #iterate over the classes, so 1,2,3...7
            for className, classFreq in self.feature_class_value_counts.items():
                newList[className] = {}
                #iterate over the columns in dfCopy  
                for col in dfCopy:
                    binValue = row[col]
                    
                    #We want only the probability from the feature_class_value_counts that equals the right className, column and bin 
                    #Thats why we put that in another dictionary with just that probability for that class
                    #And it is easier to do the product in the next loop :)
                    
                    #Using the bin value as an index is not good, since not all columns have the max number of bins
                    #so we get the proper indexes instead
                    binValueIndex = -1
                    for idx in range(len(self.feature_class_value_counts[className][col].keys())):
                        if (self.feature_class_value_counts[className][col].keys()[idx] == binValue):
                            binValueIndex = idx
                            break
                    
                    newList[className][col] = self.feature_class_value_counts[className][col][binValueIndex]
                
#
# Hint 3: Calculate the non-normalized estimated class probabilities by multiplying the class priors to the
#         product of the relative frequencies

            for className, classFreq in newList.items():
                class_prob[className] = self.class_priors[className] * np.prod(list(classFreq.values()))
                
#         
# Hint 4: Normalize the probabilities by dividing by the sum of the non-normalized probabilities; in case
#         this sum is zero, then set the probabilities to the class priors

            probability_sum = sum(class_prob.values())
            
            for className in class_prob.keys():
                if probability_sum == 0:
                    class_prob[className] = self.class_priors[className]
                else:
                    class_prob[className] = class_prob[className] / probability_sum
            dfResults = dfResults.append(class_prob, ignore_index=True)
            
        return dfResults
            

        

In [136]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.csv")

glass_test_df = pd.read_csv("glass_test.csv")

nb_model = NaiveBayes()

test_labels = glass_test_df["CLASS"]

nobins_values = [3,5,10]
bintype_values = ["equal-width","equal-size"]
parameters = [(nobins,bintype) for nobins in nobins_values for bintype in bintype_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    nb_model.fit(glass_train_df,nobins=parameters[i][0],bintype=parameters[i][1])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = nb_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([nobins_values,bintype_values]),
                       columns=["Accuracy","Brier score","AUC"])

print()
display("results",results)

Training time (3, 'equal-width'): 0.11 s.
Testing time (3, 'equal-width'): 0.39 s.
Training time (3, 'equal-size'): 0.13 s.
Testing time (3, 'equal-size'): 0.43 s.
Training time (5, 'equal-width'): 0.11 s.
Testing time (5, 'equal-width'): 0.40 s.
Training time (5, 'equal-size'): 0.12 s.
Testing time (5, 'equal-size'): 0.40 s.
Training time (10, 'equal-width'): 0.12 s.
Testing time (10, 'equal-width'): 0.48 s.
Training time (10, 'equal-size'): 0.11 s.
Testing time (10, 'equal-size'): 0.50 s.



'results'

Accuracy  Brier score       AUC
3  equal-width  0.616822     0.622116  0.724335
   equal-size   0.607477     0.554782  0.780163
5  equal-width  0.644860     0.551101  0.771688
   equal-size   0.598131     0.581556  0.796675
10 equal-width  0.654206     0.516379  0.817839
   equal-size   0.588785     0.741668  0.751165

In [137]:
train_labels = glass_train_df["CLASS"]
nb_model.fit(glass_train_df)
predictions = nb_model.predict(glass_train_df)
print("Accuracy on training set: {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.4f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.4f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 0.8505
AUC on training set: 0.9687
Brier score on training set: 0.2263


### Comment on assumptions, things that do not work properly, etc.